<a href="https://colab.research.google.com/github/ClaFlorez/Machine_Learning_Simplifie/blob/main/11_1_Naive_Bayes_pour_classification_de_texte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Naive Bayes pour classification de texte
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split # Import train_test_split
from sklearn.metrics import classification_report # Import classification_report
import pandas as pd
import numpy as np # Import numpy

# Démonstration Naive Bayes sur classification de texte
print("NAIVE BAYES - CLASSIFICATION DE TEXTE")
print("Objectif: Classifier automatiquement des articles de news")
print("=" * 70)

# Simuler un dataset de classification d'emails/articles
categories = ['tech', 'sport', 'politique', 'economie']
print(f"Catégories à classifier: {categories}")

# Simuler des textes (en réalité, vous utiliseriez de vrais articles)
textes_exemples = {
    'tech': [
        "intelligence artificielle machine learning algorithme",
        "smartphone application mobile technologie innovation",
        "ordinateur processeur performance calcul",
        "internet cybersécurité données protection",
        "robotique automatisation industrie futur"
    ],
    'sport': [
        "football match équipe victoire championnat",
        "tennis tournoi joueur performance classement",
        "basketball NBA finale playoffs",
        "cyclisme course étape montagne sprint",
        "natation piscine record chronométrage"
    ],
    'politique': [
        "gouvernement ministre décision politique réforme",
        "élection candidat vote campagne démocratie",
        "parlement débat loi amendement adoption",
        "président déclaration discours nation",
        "parti opposition majorité alliance"
    ],
    'economie': [
        "entreprise bénéfice chiffre affaires croissance",
        "bourse action investissement marché financier",
        "banque crédit taux intérêt inflation",
        "commerce international export import",
        "emploi chômage salaire économie travail"
    ]
}

# Créer le dataset d'entraînement
textes_train = []
labels_train = []

for categorie, exemples in textes_exemples.items():
    for exemple in exemples:
        # Répéter chaque exemple avec variations
        for _ in range(20):  # 20 variations par exemple
            # Ajouter du bruit et des variations
            mots = exemple.split()
            np.random.shuffle(mots)
            texte_varie = ' '.join(mots[:np.random.randint(3, len(mots)+1)])

            textes_train.append(texte_varie)
            labels_train.append(categorie)

print(f"Dataset de texte créé:")
print(f"  • Nombre de textes: {len(textes_train)}")
print(f"  • Catégories: {len(set(labels_train))}")

# Distribution des classes
for cat in categories:
    count = labels_train.count(cat)
    print(f"  • {cat}: {count} textes ({count/len(labels_train)*100:.1f}%)")

# Vectorisation TF-IDF
print(f"\nVectorisation TF-IDF des textes:")
print("=" * 40)

vectorizer = TfidfVectorizer(
    max_features=1000,     # Limiter le vocabulaire
    stop_words='english',  # Supprimer les mots vides
    ngram_range=(1, 2)     # Mots simples et bigrammes
)

X_tfidf = vectorizer.fit_transform(textes_train)
print(f"  • Vocabulaire: {len(vectorizer.get_feature_names_out())} mots")
print(f"  • Matrice TF-IDF: {X_tfidf.shape}")
print(f"  • Densité: {X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1]):.1%}")

# Diviser les données
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    X_tfidf, labels_train, test_size=0.2, random_state=42, stratify=labels_train
)

# Entraîner Naive Bayes Multinomial (optimal pour texte)
nb_model = MultinomialNB(alpha=1.0)  # Lissage de Laplace
nb_model.fit(X_train_text, y_train_text)

# Évaluer
accuracy_nb = nb_model.score(X_test_text, y_test_text)
y_pred_nb = nb_model.predict(X_test_text)

print(f"\nPerformance Naive Bayes sur texte:")
print(f"  • Accuracy: {accuracy_nb:.4f} ({accuracy_nb:.1%})")

# Rapport détaillé
print(f"\nRapport de classification détaillé:")
print(classification_report(y_test_text, y_pred_nb))

# Analyser les mots les plus discriminants
print(f"\nMots les plus discriminants par catégorie:")
print("=" * 60)

feature_names = vectorizer.get_feature_names_out()

for i, categorie in enumerate(nb_model.classes_):
    # Récupérer les log-probabilités des features pour cette classe
    log_probs = nb_model.feature_log_prob_[i]

    # Top 10 mots pour cette catégorie
    top_indices = np.argsort(log_probs)[-10:]
    top_words = [feature_names[idx] for idx in top_indices]
    top_probs = [log_probs[idx] for idx in top_indices]

    print(f"\n{categorie.upper()}:")
    for word, prob in zip(reversed(top_words), reversed(top_probs)):
        print(f"  • {word}: {prob:.3f}")

# Tester sur de nouveaux textes
print(f"\nTest sur de nouveaux textes:")
print("=" * 40)

nouveaux_textes = [
    "algorithme apprentissage automatique intelligence",
    "match football équipe victoire",
    "élection président vote démocratie",
    "entreprise bourse investissement profit"
]

nouveaux_textes_tfidf = vectorizer.transform(nouveaux_textes)
predictions_nouveaux = nb_model.predict(nouveaux_textes_tfidf)
probabilities_nouveaux = nb_model.predict_proba(nouveaux_textes_tfidf)

for i, (texte, prediction, probs) in enumerate(zip(nouveaux_textes, predictions_nouveaux, probabilities_nouveaux)):
    print(f"\nTexte {i+1}: '{texte}'")
    print(f"  • Prédiction: {prediction}")
    print(f"  • Confiance: {probs.max():.1%}")
    print(f"  • Probabilités par classe:")

    for j, (classe, prob) in enumerate(zip(nb_model.classes_, probs)):
        print(f"    - {classe}: {prob:.1%}")

NAIVE BAYES - CLASSIFICATION DE TEXTE
Objectif: Classifier automatiquement des articles de news
Catégories à classifier: ['tech', 'sport', 'politique', 'economie']
Dataset de texte créé:
  • Nombre de textes: 400
  • Catégories: 4
  • tech: 100 textes (25.0%)
  • sport: 100 textes (25.0%)
  • politique: 100 textes (25.0%)
  • economie: 100 textes (25.0%)

Vectorisation TF-IDF des textes:
  • Vocabulaire: 419 mots
  • Matrice TF-IDF: (400, 419)
  • Densité: 1.6%

Performance Naive Bayes sur texte:
  • Accuracy: 1.0000 (100.0%)

Rapport de classification détaillé:
              precision    recall  f1-score   support

    economie       1.00      1.00      1.00        20
   politique       1.00      1.00      1.00        20
       sport       1.00      1.00      1.00        20
        tech       1.00      1.00      1.00        20

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        